Relevant Feature Extraction with SHAP
===

Avicenna uses a pattern matching approach to learn and relations between input elements. Matching all input elements (features) and assosiating their relation with the program behavior is not feasible. Thus, **Avicenna** uses a machine learning model to differentiate between passing and failing inputs (based on the input properties) and then uses [SHAP]() to extract the input elements that have the highest influence on the prediction outcome.

In the following notebook, we will demonstrate how **Avicenna** extracts the most important input elements that are most likely to cause the failure.

#### Example Subject

We use our sample calculator subject to illustrate Avicenna's capabilities. Please refere to the other notebooks, like [calculator.ipynb](./calculator.ipynb), for an in-depth walkthrough of the calculator subject.

In [1]:
from avicenna_formalizations.calculator import grammar, oracle, initial_inputs

In [2]:
for inp in initial_inputs:
    print(inp.ljust(20), oracle(inp))

cos(10)              NO_BUG
sqrt(28367)          NO_BUG
tan(-12)             NO_BUG
sqrt(-900)           BUG


In [3]:
from fuzzingbook.Parser import EarleyParser, tree_to_string

parser = EarleyParser(grammar)
for inp in initial_inputs:
    for tree in parser.parse(inp):
        assert tree_to_string(tree) == inp

## Constructing Feature Vectors

In [4]:
from avicenna.oracle import OracleResult
from avicenna.input import Input
from avicenna.monads import Exceptional
from avicenna.feature_collector import GrammarFeatureCollector

In [5]:
test_inputs = set()
for inp in initial_inputs:
    test_inputs.add(
        Input.from_str(grammar, inp, oracle=oracle(inp))
    )

for inp in test_inputs:
    print(str(type(inp)).ljust(40), str(inp).ljust(20), inp.oracle)

<class 'avicenna.input.Input'>           tan(-12)             NO_BUG
<class 'avicenna.input.Input'>           sqrt(-900)           BUG
<class 'avicenna.input.Input'>           sqrt(28367)          NO_BUG
<class 'avicenna.input.Input'>           cos(10)              NO_BUG


#### Collecting Features

In [6]:
from avicenna.feature_collector import GrammarFeatureCollector, FeatureVector
from avicenna.features import NumericFeature

collector = GrammarFeatureCollector(grammar, feature_types=[NumericFeature])

In [7]:
feature_vector: FeatureVector = collector.collect_features(
    Input.from_str(grammar, "sqrt(-900)")
)

In [8]:
print(feature_vector)

sqrt(-900): {num(<onenine>): 9.0, num(<digit>): 0.0, num(<maybe_frac>): -inf, num(<number>): -900.0, num(<digits>): 0.0, num(<maybe_digits>): 0.0}


In [9]:
from avicenna.features import (
    ExistenceFeature,
    NumericFeature,
    LengthFeature,
    DerivationFeature,
)

features = [
    ExistenceFeature,
    NumericFeature,
    DerivationFeature,
    LengthFeature
]
collector = GrammarFeatureCollector(grammar, feature_types=features)
feature_vector: FeatureVector = collector.collect_features(
    Input.from_str(grammar, "sqrt(-900)")
)
print(feature_vector)

sqrt(-900): {exists(<start>): 1, exists(<arith_expr>): 1, exists(<function>): 1, exists(<number>): 1, exists(<maybe_minus>): 1, exists(<onenine>): 1, exists(<digit>): 1, exists(<maybe_digits>): 1, exists(<digits>): 1, exists(<maybe_frac>): 1, num(<onenine>): 9.0, num(<digit>): 0.0, num(<maybe_frac>): -inf, num(<number>): -900.0, num(<digits>): 0.0, num(<maybe_digits>): 0.0, exists(<start> -> <arith_expr>): 1, exists(<arith_expr> -> <function>(<number>)): 1, exists(<function> -> sqrt): 1, exists(<function> -> sin): 0, exists(<function> -> cos): 0, exists(<function> -> tan): 0, exists(<number> -> <maybe_minus><onenine><maybe_digits><maybe_frac>): 1, exists(<maybe_minus> -> ): 0, exists(<maybe_minus> -> -): 1, exists(<onenine> -> 1): 0, exists(<onenine> -> 2): 0, exists(<onenine> -> 3): 0, exists(<onenine> -> 4): 0, exists(<onenine> -> 5): 0, exists(<onenine> -> 6): 0, exists(<onenine> -> 7): 0, exists(<onenine> -> 8): 0, exists(<onenine> -> 9): 1, exists(<digit> -> 0): 1, exists(<digit> 

In [10]:
for inp in test_inputs:
    inp.update_features(collector.collect_features(inp))
    #print(inp.features)

## Using _SHAP_ to Learn Most Influential Input Propterties

In [11]:
from avicenna import feature_extractor

In [12]:
feature_learner = feature_extractor.DecisionTreeRelevanceLearner(
    grammar, prune_parent_correlation=False
)
(
    relevant_features,
    correlating_features,
    excluded_features,
) = feature_learner.learn(test_inputs)

In [13]:
print(relevant_features)

{exists(<onenine> -> 9)}


In [14]:
feature_learner = feature_extractor.SHAPRelevanceLearner(
    grammar,
    classifier_type=feature_extractor.GradientBoostingTreeRelevanceLearner,
    top_n=2,
)

In [15]:
rel, corr, ex = feature_learner.learn(test_inputs)

In [16]:
from isla.fuzzer import GrammarFuzzer
import copy

gold_standard_size = 500
reduced_size = 50

fuzzer = GrammarFuzzer(grammar)

gold_standard_inputs = set()
for _ in range(gold_standard_size):
    inp = fuzzer.fuzz_tree()
    new_input = Input(inp, oracle(inp))
    while new_input in gold_standard_inputs:
        inp = fuzzer.fuzz_tree()
        new_input = Input(inp, oracle(inp))
    gold_standard_inputs.add(new_input)

print(f"Length: {len(gold_standard_inputs)}")

small_sized_inputs = set()
for _ in range(gold_standard_size):
    inp = fuzzer.fuzz_tree()
    small_sized_inputs.add(
        Input(inp, oracle(inp))
    )

for inp in gold_standard_inputs:
    inp.update_features(collector.collect_features(inp))

for inp in small_sized_inputs:
    inp.update_features(collector.collect_features(inp))

gold_standard_inputs_copy = copy.deepcopy(gold_standard_inputs)

Length: 500


In [17]:
from avicenna.input_reducer import generate_features, select_dissimilar_inputs, create_Input_list, normalize_features, reduce_dimensionality
import re
features = generate_features(gold_standard_inputs, grammar, [ExistenceFeature, DerivationFeature, NumericFeature, LengthFeature])
input_feature_vector_list = reduce_dimensionality(features, with_PCA = True)
print(f"Original features: {len(features[0].get_features())}, after monotone removal: {len(input_feature_vector_list[0].get_features())}")

normalized_input_vector_list = normalize_features(input_feature_vector_list)

selected_input_feature_vectors = select_dissimilar_inputs(normalized_input_vector_list, reduced_size)
inputs =  create_Input_list(selected_input_feature_vectors, gold_standard_inputs)

sqrt_cnt = 0
tan_cnt = 0
sin_cnt = 0
cos_cnt = 0
negative_cnt = 0
for inp in inputs:
    inp = str(inp)
    if "sqrt" in inp:
        sqrt_cnt = sqrt_cnt + 1
    elif "tan" in inp:
        tan_cnt = tan_cnt + 1
    elif "sin" in inp:
        sin_cnt = sin_cnt + 1
    elif "cos" in inp:
        cos_cnt = cos_cnt + 1
    if "-" in inp:
        negative_cnt = negative_cnt + 1
print(f"sqrt: {sqrt_cnt}, tan: {tan_cnt}, sin: {sin_cnt}, cos: {cos_cnt}, negative: {negative_cnt}")

sqrt_cnt = 0
tan_cnt = 0
sin_cnt = 0
cos_cnt = 0
negative_cnt = 0
for inp in gold_standard_inputs:
    inp = str(inp)
    if "sqrt" in inp:
        sqrt_cnt = sqrt_cnt + 1
    elif "tan" in inp:
        tan_cnt = tan_cnt + 1
    elif "sin" in inp:
        sin_cnt = sin_cnt + 1
    elif "cos" in inp:
        cos_cnt = cos_cnt + 1
    if "-" in inp:
        negative_cnt = negative_cnt + 1
print(f"sqrt: {sqrt_cnt}, tan: {tan_cnt}, sin: {sin_cnt}, cos: {cos_cnt}, negative: {negative_cnt}")

BEFORE PCA: (500, 43)
AFTER PCA: (500, 34)


AttributeError: 'numpy.ndarray' object has no attribute 'get_features'

In [ ]:
feature_learner = feature_extractor.SHAPRelevanceLearner(
    grammar,
    classifier_type=feature_extractor.GradientBoostingTreeRelevanceLearner,
    top_n=2,
    show_beeswarm_plot=True
)

In [ ]:
relevant_features_gs, correlating_features_gs, _ = feature_learner.learn(gold_standard_inputs)
relevant_features_rs, correlating_features_rs, _ = feature_learner.learn(small_sized_inputs)
relevant_features_ds, correlating_features_ds, _ = feature_learner.learn(inputs)

In [ ]:
relevant_features_gs

In [ ]:
relevant_features_rs

In [ ]:
relevant_features_ds

In [ ]:
correlating_features_gs

In [ ]:
correlating_features_rs

In [ ]:
correlating_features_ds